# Chapter 8: Credit Risk and Credit Default Swap 

In [3]:
# First method
promised_pmt = 1000
expected_pmt = 1000*0.8+600*0.2
# EL is the difference between promosed and expected payments
EL = promised_pmt - expected_pmt
print("the expected loss is", EL)

# Second method
EAD = 1000
PD = 0.2
LGD = 0.4
print(f"EL = EAD*PD*LGD = {EAD}*{PD}*{LGD} =",EAD*PD*LGD)

the expected loss is 80.0
EL = EAD*PD*LGD = 1000*0.2*0.4 = 80.0


In [4]:
EAD = 300000 * 80
PD = 0.2
LGD = 1-0.45
print(f"EL = EAD*PD*LGD = {EAD}*{PD}*{LGD} =",EAD*PD*LGD)


EL = EAD*PD*LGD = 24000000*0.2*0.55 = 2640000.0


In [6]:
# Get current assets and current liabilities from you
CA = float(input("What's current assets (in $m)?\n"))
CL = float(input("What's current liabilities (in $m)?\n"))
# Get total assets and retained earnings from you
TA = float(input("What's total assets (in $m)?\n"))
RR = float(input("What's retained earnings (in $m)?\n"))
# Get EBIT and market capitalization from you
EBIT = float(input("What's EBIT (in $m)?\n"))
mkt_cap = float(input("What's market capitalization (in $m)?\n"))
# Get total liabilities and sales from you
TL = float(input("What's total liabilities (in $m)?\n"))
SALES = float(input("What's total sales (in $m)?\n"))

# Calculate A, B, C, D, and E
A = (CA-CL)/TA
B = RR/TA
C = EBIT/TA
D = mkt_cap/TL
E = SALES/TA

# Calcualte the Z-score and print out
Z = 1.2*A + 1.4*B + 3.3*C + 0.6*D + E
print("the Z-score is", Z)

# Predict default based on Z-score
if Z>3.0:
    print("the company is unlikely to default in the next year")
elif Z>2.7:
    print("the company is on alert")
elif Z>1.8:
    print("the company is likely to default in the next year")
else:
    print("the company is highly likely to default in the next year")

What's current assets (in $m)?
26717
What's current liabilities (in $m)?
14248
What's total assets (in $m)?
52148
What's retained earnings (in $m)?
-5399
What's EBIT (in $m)?
1994
What's market capitalization (in $m)?
593871
What's total liabilities (in $m)?
28418
What's total sales (in $m)?
31536
the Z-score is 13.411531285278544
the company is unlikely to default in the next year


In [7]:

from scipy.stats import norm
from numpy import exp, log, sqrt
import numpy as np
from scipy.optimize import minimize

S0 = 3
std_s = 0.80
r = 0.05
T = 1
D = 10


def d1(s,r,t,x,std):
    return (log(s/x)+(r+0.5*std**2)*t)/(std*sqrt(t))

def d2(s,r,t,x,std):
    return (log(s/x)+(r-0.5*std**2)*t)/(std*sqrt(t))
    
def equations(z):
    std_v = z[0]
    V0 = z[1]

    f = np.zeros(2)
    f[0] = S0-V0*norm.cdf(d1(V0,r,T,D,std_v))+D*exp(-r*T)*norm.cdf(d2(V0,r,T,D,std_v))
    f[1] = std_s*S0-norm.cdf(d1(V0,r,T,D,std_v))*std_v*V0
    return np.dot(f,f)

def cons_x(z):
    std_v = z[0]
    V0 = z[1]
    
    f = np.zeros(3)
    f[0] = V0-S0
    f[1] = S0+D-V0
    f[2] = std_v
    return f

cons = {'type' : 'ineq', 'fun': cons_x}
res = minimize(equations, (std_s, S0), method='SLSQP', constraints=cons)

print(res)

std_v=res.x[0]
V0=res.x[1]
print("std_v is", std_v)
print("V0 is", V0)
print("d2 is", d2(V0,r,T,D,std_v))
print("prob of default is", norm.cdf(-d2(V0,r,T,D,std_v)))

     fun: 2.4183249524966588e-08
     jac: array([-0.00102973,  0.0001767 ])
 message: 'Optimization terminated successfully'
    nfev: 18
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([ 0.21228832, 12.39556527])
std_v is  0.21228831687082533
V0 is  12.395565267315597
d2  1.1409978407971217
prob of default is 0.12693541076442616


In [3]:
!pip install numpy_financial
import numpy_financial as npf

# Calculate expected cash flow and discount rate
expected_pmt = 1000*0.8+600*0.2
rf = 0.06
# Calculate bond price
P_unsystematic = expected_pmt/(1+rf)
print(f"If the credit risk is unsystematic, the bond price is", P_unsystematic)
# Calculate yield to maturity
ymt_unsystematic = npf.irr([-P_unsystematic, 1000])
print(f"Yield to maturity of the bond price is", ymt_unsystematic)
# Calculate credit spread
credit_spread_unsystematic = ymt_unsystematic - rf
print(f"credit spread is", credit_spread_unsystematic)

If the credit risk is unsystematic, the bond price is 867.9245283018868
Yield to maturiy of the bond price is 0.15217391304347827
credit spread is 0.09217391304347827


In [5]:
# Obtain the risk premium
risk_premium = 0.04
# Calculate bond price
P_systematic = expected_pmt/(1+rf+risk_premium)
print(f"If the credit risk is systematic, the bond price is", P_systematic)
# Calculate yield to maturity
ymt_systematic = npf.irr([-P_systematic, 1000])
print(f"Yield to maturity of the bond price is", ymt_systematic)
# Calculate credit spread
credit_spread_systematic = ymt_systematic - rf
print(f"credit spread is", credit_spread_systematic)


If the credit risk is systematic, the bond price is 836.3636363636363
Yield to maturity of the bond price is 0.19565217391304346
credit spread is 0.13565217391304346


In [6]:
# First calculate the risk-free bond price
P_rf = 1000/(1+rf)
# The CDS price if credit risk is unsystematic
CDS_unsystematic = P_rf - P_unsystematic
print(f"If the credit risk is unsystematic, CDS price is", CDS_unsystematic)
# The CDS price if credit risk is systematic
CDS_systematic = P_rf - P_systematic
print(f"If the credit risk is systematic, CDS price is", CDS_systematic)

If the credit risk is unsystematic, CDS price is 75.47169811320748
If the credit risk is systematic, CDS price is 107.03259005145799


Question 1:

In [1]:
# First method
promised_pmt = 300000*0.8
expected_pmt = promised_pmt*0.8+0.45*promised_pmt*0.2
# EL is the difference between promosed and expected payments
EL = promised_pmt - expected_pmt
print("the expected loss is", EL)

# Second method
EAD = 300000*0.8
PD = 0.2
LGD = 1-0.45
print(f"EL = EAD*PD*LGD = {EAD}*{PD}*{LGD} =",EAD*PD*LGD)

the expected loss is 26400.0
EL = EAD*PD*LGD = 240000.0*0.2*0.55 = 26400.000000000004


Question 2:

In [2]:
NWC = 150000
TA = 1000000
Retained_Earnings = 400000
EBIT = 50000
mkt_cap = 500000
TL = 300000
SALES = 1000000

# Calculate A, B, C, D, and E
A = NWC/TA
B = Retained_Earnings/TA
C = EBIT/TA
D = mkt_cap/TL
E = SALES/TA

# Calcualte the Z-score and print out
Z = 1.2*A + 1.4*B + 3.3*C + 0.6*D + E
print("the Z-score is", Z)

# Predict default based on Z-score
if Z>3.0:
    print("the company is unlikely to default in the next year")
elif Z>2.7:
    print("the company is on alert")
elif Z>1.8:
    print("the company is likely to default in the next year")
else:
    print("the company is highly likely to default in the next year")

the Z-score is 2.9050000000000002
the company is on alert


Question 4:

In [3]:
from scipy.stats import norm
from numpy import exp, log, sqrt
import numpy as np
from scipy.optimize import minimize

S0 = 5
std_s = 0.60
r = 0.03
T = 1
D = 20


def d1(s,r,t,x,std):
    return (log(s/x)+(r+0.5*std**2)*t)/(std*sqrt(t))

def d2(s,r,t,x,std):
    return (log(s/x)+(r-0.5*std**2)*t)/(std*sqrt(t))
    
def equations(z):
    std_v = z[0]
    V0 = z[1]

    f = np.zeros(2)
    f[0] = S0-V0*norm.cdf(d1(V0,r,T,D,std_v))+D*exp(-r*T)*norm.cdf(d2(V0,r,T,D,std_v))
    f[1] = std_s*S0-norm.cdf(d1(V0,r,T,D,std_v))*std_v*V0
    return np.dot(f,f)

def cons_x(z):
    std_v = z[0]
    V0 = z[1]
    
    f = np.zeros(3)
    f[0] = V0-S0
    f[1] = S0+D-V0
    f[2] = std_v
    return f

cons = {'type' : 'ineq', 'fun': cons_x}
res = minimize(equations, (std_s, S0), method='SLSQP', constraints=cons)

print(res)

std_v=res.x[0]
V0=res.x[1]
print("std_v is", std_v)
print("V0 is", V0)
print("d2 is", d2(V0,r,T,D,std_v))
print("prob of default is", norm.cdf(-d2(V0,r,T,D,std_v)))

     fun: 3.141577682099605e-08
     jac: array([0.00487506, 0.00031606])
 message: 'Optimization terminated successfully.'
    nfev: 78
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([ 0.12717671, 24.36855877])
std_v is 0.1271767094578302
V0 is 24.36855877063606
d2 is 1.7257444076513948
prob of default is 0.042196702543413865


Question 6:

In [5]:
price_rf = 1000/(1+0.05)
print(f"the price of the risk free bond is {price_rf}")

# First method
promised_pmt = 1000
expected_pmt = promised_pmt*0.9+700*0.1
# EL is the difference between promosed and expected payments
EL = promised_pmt - expected_pmt
print("the expected loss is", EL)

# Second method
EAD = 1000
PD = 0.1
LGD = 0.3
print(f"EL = EAD*PD*LGD = {EAD}*{PD}*{LGD} =",EAD*PD*LGD)

the price of the risk free bond is 952.3809523809523
the expected loss is 30.0
EL = EAD*PD*LGD = 1000*0.1*0.3 = 30.0


In [8]:
price_risky = expected_pmt/(1+0.06)
print(f"the price of the risky bond is {price_risky}")

# Calculate YTM using the numpy_financial module
import numpy_financial as npf
YTM = npf.rate(1, 0, price_risky, -1000, 0)
print(f"the yield to maturity of the bond is {YTM}")
credit_spread = YTM - 0.05
print(f"the credit spread of the bond is {credit_spread}")

the price of the risky bond is 915.0943396226414
the yield to maturity of the bond is 0.0927835051546392
the credit spread of the bond is 0.042783505154639204


Quesiton 7:

In [9]:
CDS = price_rf - price_risky
print(f"the price of the CDS is {CDS}")

the price of the CDS is 37.286612758310866


Question 9:

In [10]:
print(f"the discount rate of the risky bond should be 0.05")
price_risky = expected_pmt/(1+0.05)
print(f"the price of the risky bond is {price_risky}")

# Calculate YTM using the numpy_financial module
import numpy_financial as npf
YTM = npf.rate(1, 0, price_risky, -1000, 0)
print(f"the yield to maturity of the bond is {YTM}")
credit_spread = YTM - 0.05
print(f"the credit spread of the bond is {credit_spread}")
CDS = price_rf - price_risky
print(f"the price of the CDS is {CDS}")

the discount rate of the risky bond should be 0.05
the price of the risky bond is 923.8095238095237
the yield to maturity of the bond is 0.08247422680412364
the credit spread of the bond is 0.03247422680412364
the price of the CDS is 28.571428571428555
